In [ ]:
import os
print(os.getcwd())

In [ ]:
import pandas as pd
import requests

# Загрузка данных в `ClickHouse`


In [ ]:
hits_df = pd.read_csv('metrika_cloud_case_data_hits.csv', sep = '\t')
visits_df = pd.read_csv('metrika_cloud_case_data_visits.csv', sep = '\t')

## Подключение и настройка
https://cloud.yandex.ru/docs/managed-clickhouse/
(см. слайды)

## Данные для доступа
* Из интерфейса облака в разделе хосты копируем имя хост в переменную `CH_HOST_NAME`
* Используем заведенного юзера в переменной `CH_USER`
* Используем заведенного юзера в переменной `CH_USER`

* Сохраним пароль заведенного пользователя в текстовый файл `.chpass.txt`
* В переменную `CH_PASS` считаем содержимое файла `.chpass.txt`
* В переменную `cacert` поместим путь к сертификату для подключения к серверу. Файл `YandexInternalRootCA.crt` должен лежать в репозитории

In [ ]:
#----------Вводить свои данные в нижние 3 переменные--------------
CH_HOST_NAME = 'rc1b-bbmw2gw58fm2dbeb.mdb.yandexcloud.net'
CH_USER      = 'z_sergey'
CH_DB_NAME   = 'metrica_data'

#-------------------------------------------
CH_PASS      = open('../.chpass.txt').read().strip()
CH_HOST      = f'https://{CH_HOST_NAME}:8443'
CH_CASERT    = 'YandexInternalRootCA.crt'

##  Функции для интеграции с ClickHouse

В файле `some_funcs` есть класс `simple_ch_client` для работы с ClickHouse

Сначала надо создать экземпляр класса, инициализировав его начальными параметрами - хост, пользователь, пароль и путь к сертификату
`simple_ch_client(CH_HOST, CH_USER, CH_PASS, cacert)`

В классе есть 4 метода:
* `.get_version()` - получает текущую версию ClickHouse. Хороший способ проверить, что указанные при инициализации параметры работают
* `.get_clickhouse_data(query)` - выполняет запрос `query` и возвращает результат в текстовом формате
* `.get_clickhouse_df(query)` - выполняет запрос `query` и возвращает результат в виде DataFrame
* `.upload(table, content)` - загружает таблицу `content`, которая подается в текстовом формате в таблицу ClickHouse'а с именем `table`


## Проверяем ClickHouse
Используя заговленные выше переменные проверим доступ до сервера (как в документации https://cloud.yandex.ru/docs/managed-clickhouse/operations/connect#connection-string)
Этот метод реализован в методе `.get_version()` класса для работы с ClickHouse
При успешном подключении не произойдет никакой ошибки при выполнении этого метода, и он сам вернет версию сервера ClickHouse (например `21.3.2.5`)

In [ ]:
import some_funcs
from some_funcs import simple_ch_client

In [ ]:
my_client = simple_ch_client(CH_HOST, CH_USER, CH_PASS, CH_CASERT)

In [ ]:
my_client.get_version()

### Загружаем данные

#### Хиты

In [ ]:
hits_df.head()

In [ ]:
hits_df.rename(columns={'ym:pv:browser':'Browser',
                'ym:pv:clientID':'ClientID',
                'ym:pv:date':'EventDate',
                'ym:pv:dateTime':'EventTime',
                'ym:pv:deviceCategory':'DeviceCategory',
                'ym:pv:lastTrafficSource':'TraficSource',
                'ym:pv:operatingSystemRoot':'OSRoot',
                'ym:pv:URL':'URL'}, inplace = True)

In [ ]:
q = f'drop table if exists {CH_DB_NAME}.hits '
my_client.get_clickhouse_data(q)

q = f'''
create table {CH_DB_NAME}.hits (
    Browser String,
    ClientID UInt64,
    EventDate Date,
    EventTime DateTime,
    DeviceCategory String,
    TraficSource String,
    OSRoot String,
    URL String
) ENGINE = MergeTree(EventDate, intHash32(ClientID), (EventDate, intHash32(ClientID)), 8192)
'''

my_client.get_clickhouse_data(q)

In [ ]:
my_client.upload(
    f'{CH_DB_NAME}.hits',
    hits_df.to_csv(index = False, sep = '\t'))

#### Визиты

In [ ]:
visits_df.head()

In [ ]:
visits_df.rename(columns={'ym:s:browser':'Browser',
                'ym:s:clientID':'ClientID',
                'ym:s:date':'StartDate',
                'ym:s:dateTime':'StartTime',
                'ym:s:deviceCategory':'DeviceCategory',
                'ym:s:lastTrafficSource':'TraficSource',
                'ym:s:operatingSystemRoot':'OSRoot',
                'ym:s:purchaseRevenue': 'Purchase.Revenue', 
                'ym:s:purchaseID': 'Purchase.ID',
                'ym:s:startURL':'StartURL'}, inplace = True)

In [ ]:
visits_df['Purchases'] = visits_df['Purchase.Revenue'].map(lambda x:x.count(','))
visits_df['Revenue'] = visits_df['Purchase.Revenue'].map(lambda x: sum(map(int,x[1:-1].split(','))) if x != '[]' else 0)


In [ ]:
visits_df.drop(columns=['Purchase.ID','Purchase.Revenue'], inplace=True)

In [ ]:
q = f'drop table if exists {CH_DB_NAME}.visits '
my_client.get_clickhouse_data(q)

q = f'''
create table {CH_DB_NAME}.visits (
    Browser String,
    ClientID UInt64,
    StartDate Date,
    StartTime DateTime,
    DeviceCategory UInt8,
    TraficSource String,
    OSRoot String,
    Purchases Int32,
    Revenue Double,
    StartURL String
) ENGINE = MergeTree(StartDate, intHash32(ClientID), (StartDate, intHash32(ClientID)), 8192)
'''

my_client.get_clickhouse_data(q)

In [ ]:
my_client.upload(
    f'{CH_DB_NAME}.visits',
    visits_df.to_csv(sep='\t', index =False))